In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, RocCurveDisplay
import itertools
from sklearn.model_selection import GridSearchCV
pd.options.display.float_format = '{:.2f}'.format
import warnings                   # to deal with warnings
warnings.filterwarnings('ignore')

In [2]:
# View dataframe
url='https://drive.google.com/file/d/1fC0xJhngpnQ20xenGxxXViHuqj0ifJcO/view?usp=drive_link'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url)

df = df.replace({np.nan: 'none'})
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# Drop duplicates
df.drop_duplicates(keep='first', inplace=True)

print(f'Number of duplicates: {df.duplicated().sum()}')

Number of duplicates: 0


In [4]:
# Remove extra spaces


# apply map applies function to every scalar in the dataframe
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df.replace('?', np.nan, inplace=True)

pd.isna(df).sum()[pd.isna(df).sum()>0]

workclass         1836
occupation        1843
native-country     582
dtype: int64

In [5]:
df.workclass.fillna(value='UnknCl', inplace=True)
df.occupation.fillna(value='UnknOc', inplace=True)
df['native-country'].fillna(value='Unkn_ctry' , inplace=True)

## Now check whether the modification has been made or not

pd.isna(df).sum()[pd.isna(df).sum() > 0]

Series([], dtype: int64)

In [6]:
# Prune labels

# df.replace({'category' : {'label' : 'label it is replaced with'})


df.replace({'workclass': {'State-gov': 'Govt.', 'Self-emp-not-inc': 'self_emp', 'Federal-gov': 'Govt.', 'Local-gov': 'Govt.', 'Self-emp-inc':'self_emp', 'Without-pay': 'UnknCl', 'Never-worked': 'UnknCl'}}, inplace=True)
df.replace({'marital-status': {'Married-civ-spouse': 'Married', 'Divorced': 'DASW', 'Married-spouse-absent': 'DASW', 'Separated': 'DASW', 'Married-AF-spouse':'Married', 'Widowed': 'DASW'}}, inplace=True)
df.replace({'occupation': {'Adm-clerical': 'Adminstration', 'Exec-managerial': 'Executive', 'Handlers-cleaners': 'Handlers', 'Prof-specialty': 'Profsiionals', 'Other-service' : 'UnknOc', 'Craft-repair' : 'Repairing', 'Farming-fishing' : 'Farming', 'Transport-moving':'Transportation', 'Machine-op-inspct': 'MachineOp', 'Protective-serv' : 'ProtectiveServ', 'Priv-house-serv': 'HouseServ'}}, inplace=True)
df.replace({'native-country': {'United-States': 'USA', 'South': 'SouthKorea', 'Puerto-Rico': 'PuertoRico', 'Dominican-Republic': 'DominicRep', 'Outlying-US(Guam-USVI-etc)':'OutlyingUSA', 'Trinadad&Tobago': 'Tri&Tob', 'Holand-Netherlands': 'Netherlands', 'Hong' : 'HongKong'}}, inplace=True)
df.replace({'race': {'Asian-Pac-Islander': 'APAC', 'Amer-Indian-Eskimo': 'NatAm'}}, inplace=True)
## df.replace({'salary': {'<=50K': 0, '>50K': 1}}, inplace=True)

## Checking whether the modification has been made or not

for col in df.columns:
    if df[col].dtype=='object':
        print()
        print(col)
        print(df[col].unique())


workclass
['Govt.' 'self_emp' 'Private' 'UnknCl']

education
['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']

marital-status
['Never-married' 'Married' 'DASW']

occupation
['Adminstration' 'Executive' 'Handlers' 'Profsiionals' 'UnknOc' 'Sales'
 'Repairing' 'Transportation' 'Farming' 'MachineOp' 'Tech-support'
 'ProtectiveServ' 'Armed-Forces' 'HouseServ']

relationship
['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']

race
['White' 'Black' 'APAC' 'NatAm' 'Other']

sex
['Male' 'Female']

native-country
['USA' 'Cuba' 'Jamaica' 'India' 'Unkn_ctry' 'Mexico' 'SouthKorea'
 'PuertoRico' 'Honduras' 'England' 'Canada' 'Germany' 'Iran' 'Philippines'
 'Italy' 'Poland' 'Columbia' 'Cambodia' 'Thailand' 'Ecuador' 'Laos'
 'Taiwan' 'Haiti' 'Portugal' 'DominicRep' 'El-Salvador' 'France'
 'Guatemala' 'China' 'Japan' 'Yugoslavia' 'Peru' 'OutlyingUSA' 'Scotl

In [7]:
# separating features into numerical and categorical
num_df = df[['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
cat_df = df[['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex','native-country', 'salary']]

In [8]:
# label encoding

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in cat_df:
  cat_df[i] = le.fit_transform(df[i])

# Join encoded data to numeric data

main_df = pd.concat([num_df, cat_df], axis=1)

In [9]:
minmaxsca = MinMaxScaler() # Normalization
stdsca = StandardScaler() # Standardization

main_df['marital-status'] = stdsca.fit_transform(main_df[['marital-status']])
main_df['sex'] = stdsca.fit_transform(main_df[['sex']])
main_df['relationship'] = stdsca.fit_transform(main_df[['relationship']])
main_df['hours-per-week'] = stdsca.fit_transform(main_df[['hours-per-week']])
main_df['age'] = stdsca.fit_transform(main_df[['age']])
main_df['education'] = stdsca.fit_transform(main_df[['education']])
main_df['occupation'] = stdsca.fit_transform(main_df[['occupation']])
main_df['education-num'] = stdsca.fit_transform(main_df[['education-num']])

main_df['workclass'] = minmaxsca.fit_transform(main_df[['workclass']])
main_df['capital-gain'] = minmaxsca.fit_transform(main_df[['capital-gain']])
main_df['capital-loss'] = minmaxsca.fit_transform(main_df[['capital-loss']])
main_df['native-country'] = minmaxsca.fit_transform(main_df[['native-country']])
main_df['race'] = stdsca.fit_transform(main_df[['race']])

main_df.head()

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,workclass,education,education-num,marital-status,occupation,relationship,race,sex,native-country,salary
0,0.03,77516,0.02,0.00,-0.04,0.00,-0.34,1.13,1.22,-1.66,-0.28,0.40,0.70,0.93,0
1,0.84,83311,0.00,0.00,-2.22,1.00,-0.34,1.13,-0.16,-1.20,-0.90,0.40,0.70,0.93,0
2,-0.04,215646,0.00,0.00,-0.04,0.33,0.18,-0.42,-1.54,-0.73,-0.28,0.40,0.70,0.93,0
3,1.06,234721,0.00,0.00,-0.04,0.33,-2.40,-1.20,-0.16,-0.73,-0.90,-2.31,0.70,0.93,0
4,-0.78,338409,0.00,0.00,-0.04,0.33,-0.34,1.13,-0.16,-0.04,2.21,-2.31,-1.42,0.10,0


In [10]:
# Separate into dependent and independent variables

X = main_df.drop(columns=["salary", "fnlwgt", "education", "workclass", "native-country", "occupation", "marital-status"])
y = main_df["salary"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [12]:
# For Random Forest

param_grid_RFC = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

grid_RFC = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid_RFC,
                    refit=True,
                    verbose=3,
                    cv = 3,
                    scoring='accuracy',  
                    n_jobs=-1)  

grid_RFC.fit(X_train, y_train)

print(grid_RFC.best_params_)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
{'bootstrap': False, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [13]:
# For SVC

param_grid_SVC = {'C': [0.1, 0.5, 1],
                  'kernel': ['linear', 'rbf'],
                  'gamma' :['scale','auto']}

grid_SVC = GridSearchCV(SVC(),
                    param_grid_SVC,
                    refit=True,
                    cv = 3,
                    verbose=3,
                    scoring = 'accuracy',
                    n_jobs = -1)
grid_SVC.fit(X_train, y_train)

print(grid_SVC.best_params_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


In [14]:
# For Linear regression

param_grid_LR = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga'],
    'max_iter': [50, 100, 200, 300]
}

grid_LR = GridSearchCV(LogisticRegression(),
                    param_grid_LR,
                    refit=True,
                    cv = 3,
                    verbose=3,
                    scoring = 'accuracy',
                    n_jobs = -1)

grid_LR.fit(X_train, y_train)

print(grid_LR.best_params_)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
{'C': 1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}


In [15]:
# XGBoost

param_grid_XG = {
    'n_estimators': [50, 75, 100],
    'learning_rate': [0.01, 0.025, 0.5, 0.1, 0.15],
    'max_depth': [7, 10, 12],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0, 1.2],
    'gamma': [0.1, 0.2],
}

grid_XG = GridSearchCV(XGBClassifier(),
                    param_grid_XG,
                    refit=True,
                    cv = 3,
                    verbose=3,
                    scoring='accuracy',  # Optimize for accuracy
                    n_jobs = -1)  # Use all available cores

grid_XG.fit(X_train, y_train)

print(grid_XG.best_params_)

Fitting 3 folds for each of 1080 candidates, totalling 3240 fits
{'colsample_bytree': 0.9, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.9}


In [16]:
num_models = 3

RFCS = []
for i in range(num_models):
  RFC = RandomForestClassifier(**grid_RFC.best_params_)
  RFC.fit(X_train, y_train)
  RFCS.append(RFC)

SVCS = []
for i in range(num_models):
  classifier = SVC(**grid_SVC.best_params_)
  classifier.fit(X_train, y_train)
  SVCS.append(classifier)

LRS = []
for i in range(num_models):
  LR = LogisticRegression(**grid_LR.best_params_)
  LR.fit(X_train, y_train)
  LRS.append(LR)

# Does changing the hyperparameters here make any real difference to the fairness?
XGBS = []
for i in range(num_models):
  XGB = XGBClassifier(**grid_XG.best_params_)
  XGB.fit(X_train, y_train)
  XGBS.append(XGB)

In [17]:
predictions = []
for i in range(num_models):
  predictions.append(RFCS[i].predict(X_test))
  predictions.append(SVCS[i].predict(X_test))
  predictions.append(LRS[i].predict(X_test))
  predictions.append(XGBS[i].predict(X_test))

accuracy_scores = [accuracy_score(y_test, pred) for pred in predictions]

# number of columns of accuracy matrix = number of type of models
col_dim = int(len(accuracy_scores)/num_models)

# Place accuracy scores into matrix
accuracy_matrix = np.array(accuracy_scores).reshape(num_models, col_dim)
print(accuracy_matrix)

[[0.85617701 0.82928703 0.82037492 0.86432084]
 [0.85648433 0.82928703 0.82037492 0.86432084]
 [0.85510141 0.82928703 0.82037492 0.86432084]]


In [18]:
# Reduce dimension by taking average

average_accuracies = np.mean(accuracy_matrix, axis=0)
print(average_accuracies)

[0.85592092 0.82928703 0.82037492 0.86432084]


In [19]:
from sklearn.metrics import accuracy_score, precision_score
from fairlearn.metrics import (
    MetricFrame,
    count,
    false_negative_rate,
    false_positive_rate,
    selection_rate,
)

In [ ]:
sens_feature = X_test['race']

# Select metrics
metrics = {
    "false positive rate": false_positive_rate
}
metrics_list_FPRS = []
for i in range(len(predictions)):

    FPRS = MetricFrame(
        metrics=metrics,
        y_true=y_test,
        y_pred=predictions[i],
        sensitive_features = sens_feature,
    )
    metrics_list_FPRS.append(FPRS.by_group.values)

In [21]:
metrics = {
    "false_negative_rate": false_negative_rate
}
metrics_list_FNRS = []
for i in range(len(predictions)):
    FNRS = MetricFrame(
        metrics = metrics,
        y_true = y_test,
        y_pred = predictions[i],
        sensitive_features = sens_feature 
    )
    metrics_list_FNRS.append(FNRS.by_group.values)

In [22]:
FPRS_matrix = np.array(metrics_list_FPRS).reshape(num_models,col_dim,FPRS.by_group.values.size)
FNRS_matrix = np.array(metrics_list_FNRS).reshape(num_models,col_dim,FNRS.by_group.values.size)  
# first dimension is the umber of sub arrays, equal to the number of models
# second dimension is the number of model types
# third dimension is the number of features in the FPR calculation, i.e. male/female


In [39]:
# Reshape the array so each sub array corresponds to one model type
grouped_rows_FPRS = [FPRS_matrix[:, i, :] for i in range(FPRS_matrix.shape[1])]
# Stack the grouped rows into a single 3D array
combined_array_FPRS = np.stack(grouped_rows_FPRS, axis=0)
#print(combined_array_FPRS)
# Mean of combined array
average_FPRS = np.mean(combined_array_FPRS, axis = 1)
#average_FPRS = np.ravel(average_FPRS)
print(average_FPRS.T)

[[0.08888889 0.1037037  0.05185185 0.07407407]
 [0.00929368 0.00929368 0.02230483 0.01486989]
 [0.01886792 0.03773585 0.01886792 0.01886792]
 [0.04347826 0.04347826 0.04347826 0.02173913]
 [0.04038708 0.05662188 0.06238004 0.06645873]]


In [40]:
# Reshape the array so each sub array corresponds to one model type
grouped_rows_FNRS = [FNRS_matrix[:, i, :] for i in range(FNRS_matrix.shape[1])]
# Stack the grouped rows into a single 3D array
combined_array_FNRS = np.stack(grouped_rows_FNRS, axis=0)
#print(combined_array_FNRS)
# Mean of combined array
average_FNRS = np.mean(combined_array_FNRS, axis = 1)
print(average_FNRS.T)

[[0.41843972 0.4893617  0.63829787 0.36170213]
 [0.58333333 0.73611111 0.68055556 0.47222222]
 [0.22222222 0.66666667 0.33333333 0.16666667]
 [0.8        1.         1.         0.8       ]
 [0.47450162 0.53337969 0.55702364 0.36856745]]


Examine the following arrays.

Each column represents the accuracy score of a model. The first column is the score for Random Forest, the second column is the score for SVM, the third column is the score for Linear Regression and the fourth column is for XGBoost.

[0.85592092 0.82928703 0.82037492 0.86432084]

The next array contains the false positive rates (FPRs) for 5 different groups, group 1, group 2, group 3, group 4 and group 5. Each column The first column is the FPR for Random Forest, the second column is the FPR for SVM, the third column is the FPR for Linear Regression and the fourth column is the FPR for XGBoost. Each row represents one of the 5 groups.

[[0.08888889 0.1037037  0.05185185 0.07407407]
 [0.00929368 0.00929368 0.02230483 0.01486989]
 [0.01886792 0.03773585 0.01886792 0.01886792]
 [0.04347826 0.04347826 0.04347826 0.02173913]
 [0.04038708 0.05662188 0.06238004 0.06645873]]

The next array contains the false negative rates (FNRs) for 5 different groups, group 1, group 2, group 3, group 4 and group 5. Each column The first column is the FNR for Random Forest, the second column is the FNR for SVM, the third column is the FNR for Linear Regression and the fourth column is the FNR for XGBoost. Each row represents one of the 5 groups.


[[0.41843972 0.4893617  0.63829787 0.36170213]
 [0.58333333 0.73611111 0.68055556 0.47222222]
 [0.22222222 0.66666667 0.33333333 0.16666667]
 [0.8        1.         1.         0.8       ]
 [0.47450162 0.53337969 0.55702364 0.36856745]]

Do not perform any calculations or write code. Using the information in the arrays, which model provides the best trade-off between accuracy and fairness based on the FPRs and FNRs? 